<a href="https://colab.research.google.com/github/maarut001/aiml/blob/main/GRAD_50400_Programming_Project_1_KNN_Classifier_DeepaSingh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import warnings
import os # For checking file existence
# Define the expected number of features (assuming the last column is the label)
# These will now be determined dynamically within the KNNClassifier's train method.


In [3]:
import pandas as pd

# Load the CSV file into a pandas DataFrame
df = pd.read_csv('Prog1data.csv')

### CSV Description

In [4]:
# Display information about the DataFrame
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999 entries, 0 to 1998
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   7       1999 non-null   int64 
 1   7.1     1999 non-null   int64 
 2   0       1999 non-null   int64 
 3   19      1999 non-null   int64 
 4   8       1999 non-null   int64 
 5   11      1999 non-null   int64 
 6   3       1999 non-null   int64 
 7   78      1999 non-null   int64 
 8   5       1999 non-null   int64 
 9   6       1999 non-null   int64 
 10  9       1999 non-null   int64 
 11  +       1999 non-null   object
dtypes: int64(11), object(1)
memory usage: 187.5+ KB


None

### First 5 Rows

In [5]:
# Display the first 5 rows of the DataFrame
display(df.head())

,7,7.1,0,19,8,11,3,78,5,6,9,+
0,8,9,0,26,10,25,7,68,2,7,10,+
1,8,8,0,23,9,15,5,70,3,7,10,+
2,11,3,6,19,8,17,6,80,2,6,10,+
3,7,7,0,19,8,11,3,78,5,6,9,+
4,7,7,0,18,8,13,4,78,5,6,9,+


In [6]:
# Count occurrences of labels '+' and '-'
label_counts = df['+'].value_counts()
print("Count of rows with label '+':", label_counts.get('+', 0))
print("Count of rows with label '-':", label_counts.get('-', 0))

Count of rows with label '+': 998
Count of rows with label '-': 1000


In [7]:
# Check for missing data
missing_data = df.isnull().sum()
print("Missing values per column:")
print(missing_data)

Missing values per column:
7      0
7.1    0
0      0
19     0
8      0
11     0
3      0
78     0
5      0
6      0
9      0
+      0
dtype: int64


In [8]:
import csv
from math import sqrt

class KNNClassifierOld:
    def __init__(self, n_neighbors=5):
        self.n_neighbors = n_neighbors
        self.X_train = None  # list of feature lists
        self.y_train = None  # list of labels

    # ---------- internal helper: load CSV ----------
    def _load_dataset(self, filename):
        X = []
        y = []
        with open(filename, 'r') as f:
            reader = csv.reader(f)
            for row in reader:
                if not row:
                    continue
                # all but last are features (floats), last is label
                features = [float(x) for x in row[:-1]]
                label = row[-1].strip()
                X.append(features)
                y.append(label)
        return X, y

    # ---------- public API: train ----------
    def train(self, filename):
        """
        Load data from CSV and store as training set.
        """
        self.X_train, self.y_train = self._load_dataset(filename)

    # ---------- internal helper: Euclidean distance ----------
    def _euclidean_distance(self, x1, x2):
        distance = 0.0
        for i in range(len(x1)):
            distance += (x1[i] - x2[i]) ** 2
        return sqrt(distance)

    # ---------- internal helper: neighbors for ONE test point ----------
    def _get_neighbors_from(self, X_ref, y_ref, x_test):
        """
        Find k nearest neighbors of x_test in the reference set (X_ref, y_ref).
        Returns the list of neighbor labels.
        """
        distances = []
        for i, x_train in enumerate(X_ref):
            dist = self._euclidean_distance(x_test, x_train)
            distances.append((i, dist))

        # sort by distance
        distances.sort(key=lambda tup: tup[1])

        # take k closest indices
        neighbor_indices = [idx for idx, _ in distances[:self.n_neighbors]]
        neighbor_labels = [y_ref[idx] for idx in neighbor_indices]
        return neighbor_labels

    # ---------- internal helper: predict label for ONE test point ----------
    def _predict_one_from(self, X_ref, y_ref, x_test):
        neighbor_labels = self._get_neighbors_from(X_ref, y_ref, x_test)
        # majority vote
        prediction = max(set(neighbor_labels), key=neighbor_labels.count)
        return prediction

    # ---------- public API: predict ----------
    def predict(self, X_test):
        """
        X_test: list of feature vectors, e.g. [[f1, f2, ..., fN], [...], ...]
        Returns: list of predicted labels.
        """
        if self.X_train is None or self.y_train is None:
            raise ValueError("Model has not been trained. Call train() first.")

        predictions = []
        for x in X_test:
            pred = self._predict_one_from(self.X_train, self.y_train, x)
            predictions.append(pred)
        return predictions

    # ---------- public API: LOOCV score ----------
    def loocv_score(self):
        """
        Perform Leave-One-Out Cross-Validation on the currently loaded training data.
        Returns (accuracy_percent, error_percent).
        """
        if self.X_train is None or self.y_train is None:
            raise ValueError("No training data loaded. Call train() first.")

        total = len(self.X_train)
        correct = 0

        # For each sample, leave it out and predict it using the rest
        for i in range(total):
            # build training subset (all except i)
            X_subset = self.X_train[:i] + self.X_train[i+1:]
            y_subset = self.y_train[:i] + self.y_train[i+1:]

            x_test = self.X_train[i]
            y_true = self.y_train[i]

            y_pred = self._predict_one_from(X_subset, y_subset, x_test)

            if y_pred == y_true:
                correct += 1

        accuracy = correct / total * 100.0
        error = 100.0 - accuracy
        return accuracy, error


In [9]:
import csv
from math import sqrt

class KNNClassifier:
    def __init__(self, n_neighbors=5):
        if n_neighbors <= 0:
            raise ValueError("n_neighbors must be a positive integer.")
        self.n_neighbors = n_neighbors
        self.X_train = None
        self.y_train = None
        self.expected_features = None  # number of features

        # NORMALIZATION: state
        self.mins = None              # per-feature minimums
        self.maxs = None              # per-feature maximums
        self._is_normalized = False   # did we call normalize()?

    # ------------------------------------------------------
    # Load dataset with robust error checking
    # ------------------------------------------------------
    def _load_dataset(self, filename):
        X = []
        y = []

        with open(filename, 'r') as f:
            reader = csv.reader(f)
            row_num = 0

            for row in reader:
                row_num += 1
                if not row:
                    print(f"Warning: Skipping empty row {row_num}")
                    continue

                # Require at least 2 columns (features + label)
                if len(row) < 2:
                    print(f"Warning: Row {row_num} malformed, skipping: {row}")
                    continue

                features_raw = row[:-1]
                label = row[-1].strip()

                # Convert features to float
                try:
                    features = [float(val) for val in features_raw]
                except ValueError:
                    print(f"Warning: Non-numeric value in row {row_num}, skipping: {row}")
                    continue

                X.append(features)
                y.append(label)

        if len(X) == 0:
            raise ValueError("No valid rows found in dataset. Cannot train classifier.")

        self.expected_features = len(X[0])

        # NORMALIZATION: reset state whenever we load new data
        self.mins = None
        self.maxs = None
        self._is_normalized = False

        return X, y

    # ------------------------------------------------------
    # Public API: train
    # ------------------------------------------------------
    def train(self, filename):
        self.X_train, self.y_train = self._load_dataset(filename)

        if len(self.X_train) < self.n_neighbors:
            raise ValueError(
                f"Training set has only {len(self.X_train)} samples, "
                f"but k={self.n_neighbors}. Reduce k or provide more data."
            )

    # ------------------------------------------------------
    # NORMALIZATION: compute per-feature min and max
    # ------------------------------------------------------
    def _compute_minmax(self):
        n_samples = len(self.X_train)
        n_features = len(self.X_train[0])

        mins = [float('inf')] * n_features
        maxs = [float('-inf')] * n_features

        for row in self.X_train:
            for j in range(n_features):
                if row[j] < mins[j]:
                    mins[j] = row[j]
                if row[j] > maxs[j]:
                    maxs[j] = row[j]

        # Avoid zero range (max == min) to prevent division by zero
        for j in range(n_features):
            if maxs[j] == mins[j]:
                maxs[j] = mins[j] + 1.0

        return mins, maxs

    # ------------------------------------------------------
    # PUBLIC API: normalize() using min–max scaling to [0, 1]
    # ------------------------------------------------------
    def normalize(self):
        """
        Min–max normalize training data to [0, 1] per feature.
        x' = (x - min) / (max - min)
        Stores mins/maxs so predict() can scale test inputs consistently.
        """
        if self.X_train is None:
            raise ValueError("No training data loaded. Call train() before normalize().")

        self.mins, self.maxs = self._compute_minmax()

        n_samples = len(self.X_train)
        n_features = len(self.X_train[0])

        for i in range(n_samples):
            for j in range(n_features):
                self.X_train[i][j] = (self.X_train[i][j] - self.mins[j]) / (self.maxs[j] - self.mins[j])

        self._is_normalized = True

    # ------------------------------------------------------
    # Internal: Euclidean distance
    # ------------------------------------------------------
    def _euclidean_distance(self, x1, x2):
        distance = 0.0
        for i in range(len(x1)):
            distance += (x1[i] - x2[i]) ** 2
        return sqrt(distance)

    # ------------------------------------------------------
    # Internal: get k nearest neighbors
    # ------------------------------------------------------
    def _get_neighbors_from(self, X_ref, y_ref, x_test):
        if len(X_ref) < self.n_neighbors:
            raise ValueError(
                f"Training subset too small: {len(X_ref)} samples but k={self.n_neighbors}"
            )

        distances = []
        for i, x_train in enumerate(X_ref):
            dist = self._euclidean_distance(x_test, x_train)
            distances.append((i, dist))

        distances.sort(key=lambda tup: tup[1])
        neighbor_indices = [idx for idx, _ in distances[:self.n_neighbors]]
        neighbor_labels = [y_ref[idx] for idx in neighbor_indices]
        return neighbor_labels

    # ------------------------------------------------------
    # Internal: predict for one test vector
    # ------------------------------------------------------
    def _predict_one_from(self, X_ref, y_ref, x_test):
        neighbors = self._get_neighbors_from(X_ref, y_ref, x_test)
        prediction = max(set(neighbors), key=neighbors.count)
        return prediction

    # ------------------------------------------------------
    # Public API: predict
    # ------------------------------------------------------
    def predict(self, X_test):
        if self.X_train is None:
            raise ValueError("Model not trained. Call train(filename) first.")

        predictions = []
        for row_num, x in enumerate(X_test, start=1):

            # Check feature count
            if len(x) != self.expected_features:
                raise ValueError(
                    f"Test sample #{row_num} has {len(x)} features, "
                    f"but expected {self.expected_features}"
                )

            # Ensure numeric
            try:
                x = [float(val) for val in x]
            except ValueError:
                raise ValueError(
                    f"Test sample #{row_num} contains non-numeric values: {x}"
                )

            # NORMALIZATION: apply same min–max scaling to test sample
            if self._is_normalized:
                for j in range(len(x)):
                    x[j] = (x[j] - self.mins[j]) / (self.maxs[j] - self.mins[j])

            pred = self._predict_one_from(self.X_train, self.y_train, x)
            predictions.append(pred)

        return predictions

    # ------------------------------------------------------
    # Public API: LOOCV (for error estimation)
    # ------------------------------------------------------
    def loocv_score(self):
        if self.X_train is None or self.y_train is None:
            raise ValueError("No training data loaded. Call train() first.")

        total = len(self.X_train)
        correct = 0

        for i in range(total):
            X_subset = self.X_train[:i] + self.X_train[i+1:]
            y_subset = self.y_train[:i] + self.y_train[i+1:]

            x_test = self.X_train[i]
            y_true = self.y_train[i]

            if len(X_subset) < self.n_neighbors:
                raise ValueError(
                    f"During LOOCV, subset size {len(X_subset)} < k={self.n_neighbors}."
                )

            y_pred = self._predict_one_from(X_subset, y_subset, x_test)

            if y_pred == y_true:
                correct += 1

        accuracy = (correct / total) * 100.0
        error = 100.0 - accuracy
        return accuracy, error


In [10]:
# KNNClassifier is the class implemented by you
cl = KNNClassifier(n_neighbors=5)
#cl = KNNClassifier(n_neighbors=5)
cl.train("Prog1data.csv")
print(cl.predict([[2, 3, 2, -1, 5, 6, 1, 7, 8, 2, 0]]))
# Expected output: ['+']

['-']


“We apply min–max normalization using normalize() so that each feature lies in [0, 1], preventing attributes with larger numeric ranges from dominating the Euclidean distance used in KNN.”
“During prediction, all test samples are normalized using the same per-feature min and max values computed from the training data, ensuring that distance calculations occur in the same feature space. This prevents skewing and ensures consistent KNN behavior.”

Error Metric

For this project, the error metric used is the classification error rate, defined as:

Error Rate
= # of misclassified samples/
Total number of samples

The corresponding accuracy is:

Error Rate
Accuracy=1-Error Rate

How Error is Calculated

We use Leave-One-Out Cross-Validation (LOOCV) to estimate the true generalization error of the KNN classifier.

LOOCV works as follows:

For each row i in the dataset:

Remove row i from the dataset.

Train the KNN model on all remaining rows.

Use the model to predict the class label of row i.

Count how many predictions are incorrect.

Since each row is used exactly once as a test example, LOOCV provides an unbiased and stable error estimate, especially for small datasets.

In [11]:
# Get LOOCV error/accuracy for your report (Section 3.5)
acc, err = cl.loocv_score()
print(f"Leave-One-Out Cross-Validation Accuracy: {acc:.2f}%")
print(f"Leave-One-Out Cross-Validation Error: {err:.2f}%")

Leave-One-Out Cross-Validation Accuracy: 98.30%
Leave-One-Out Cross-Validation Error: 1.70%


**Discussion Points**
Naive KNN scales linearly with dataset size and feature dimension.

For large datasets (>500k rows), naive Python KNN becomes too slow.128 GB RAM allows storing extremely large datasets:
e.g., 50 million rows × 15 features fits comfortably.

KD-tree, ball-tree, VP-tree significantly speed up search for medium dimensions.

HNSW and FAISS allow millisecond-level approximate nearest neighbor retrieval.

Response time under 0.5s is achievable on a 5 GHz CPU for moderate dataset sizes, but Python overhead is a bottleneck.

Feasibility on a Typical Workstation

Workstation:

5 GHz CPU

128 GB RAM

Feasibility Observations

Memory:
128 GB RAM allows storing extremely large datasets:
e.g., 50 million rows × 15 features fits comfortably.

Computation:
At 5 GHz, CPU can handle > 5 billion operations per second.
Distance computation of 50M operations may take roughly 10-40 ms depending on implementation and Python overhead.

Python caveat:
Pure Python loops are slow; C-optimized libraries (NumPy, FAISS) make huge difference.

Verdict:

Naive pure-Python KNN is feasible for datasets up to hundreds of thousands of rows.
For millions → optimized Python datastructures.